In [4]:
import ffmpeg
import os

def get_video_quality_details(file_path):
    """
    Retrieves comprehensive video and audio quality details from a video file using ffprobe.

    Args:
        file_path (str): The path to the video file.

    Returns:
        dict: A dictionary containing detailed quality information, or None if an error occurs.
    """
    try:
        probe = ffmpeg.probe(file_path)
        video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
        audio_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'audio'), None)

        details = {
            "filename": os.path.basename(file_path),
            "format": probe['format']['format_name'] if 'format' in probe and 'format_name' in probe['format'] else "N/A",
            "duration": float(probe['format']['duration']) if 'format' in probe and 'duration' in probe['format'] else "N/A",
            "overall_bitrate": int(probe['format']['bit_rate']) if 'format' in probe and 'bit_rate' in probe['format'] else "N/A",
            "video": {
                "codec": video_stream['codec_name'] if video_stream and 'codec_name' in video_stream else "N/A",
                "profile": video_stream['profile'] if video_stream and 'profile' in video_stream else "N/A",
                "resolution": f"{video_stream['width']}x{video_stream['height']}" if video_stream and 'width' in video_stream and 'height' in video_stream else "N/A",
                "aspect_ratio": video_stream['display_aspect_ratio'] if video_stream and 'display_aspect_ratio' in video_stream else "N/A",
                "frame_rate": video_stream['avg_frame_rate'] if video_stream and 'avg_frame_rate' in video_stream else "N/A",
                "bitrate": int(video_stream['bit_rate']) if video_stream and 'bit_rate' in video_stream else "N/A",
                "scan_type": video_stream.get('field_order', 'N/A'), # 'field_order' in probe output often indicates scan type (progressive/interlaced)
                "pixel_format": video_stream.get('pix_fmt', 'N/A'),
                "color_space": video_stream.get('color_space', 'N/A'),
                "color_range": video_stream.get('color_range', 'N/A'),
                "color_transfer": video_stream.get('color_transfer', 'N/A'),
                "is_variable_rate": video_stream.get('is_variable_rate', 'N/A'),

            },
            "audio": {
                "codec": audio_stream['codec_name'] if audio_stream and 'codec_name' in audio_stream else "N/A",
                "channels": audio_stream['channels'] if audio_stream and 'channels' in audio_stream else "N/A",
                "sample_rate": int(audio_stream['sample_rate']) if audio_stream and 'sample_rate' in audio_stream else "N/A",
                "bitrate": int(audio_stream['bit_rate']) if audio_stream and 'bit_rate' in audio_stream else "N/A",
            }
        }
        return details

    except ffmpeg.Error as e:
        print(f"Error processing file: {file_path}")
        print(f"FFmpeg stderr:\n{e.stderr.decode()}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


def compare_video_quality(file1_details, file2_details):
    """
    Compares the quality details of two video files and provides a more thorough analysis
    to suggest which might be higher quality.  Includes more advanced metrics and caveats.

    Args:
        file1_details (dict): Details for the first file.
        file2_details (dict): Details for the second file.

    Returns:
        str: A detailed comparison report indicating likely quality differences and limitations.
    """
    if not file1_details or not file2_details:
        return "Could not compare quality due to missing details."

    file1_name = file1_details['filename']
    file2_name = file2_details['filename']

    comparison_report = f"Detailed Quality Comparison:\n"
    comparison_report += f"---------------------------------------------------\n"

    # Resolution Comparison
    res1 = file1_details['video']['resolution']
    res2 = file2_details['video']['resolution']
    comparison_report += f"Resolution:\n  {file1_name}: {res1}\n  {file2_name}: {res2}\n"
    res_comparison_note = ""
    if res1 != "N/A" and res2 != "N/A":
        res1_width = int(res1.split('x')[0])
        res1_height = int(res1.split('x')[1])
        res2_width = int(res2.split('x')[0])
        res2_height = int(res2.split('x')[1])
        if res1_width * res1_height > res2_width * res2_height:
            res_comparison_note = f"  -> {file1_name} has a higher resolution, which generally contributes to more detail.\n"
        elif res2_width * res2_height > res1_width * res1_height:
            res_comparison_note = f"  -> {file2_name} has a higher resolution, which generally contributes to more detail.\n"
        else:
            res_comparison_note = f"  -> Resolutions are the same.\n"
    else:
        res_comparison_note = f"  -> Unable to compare resolution.\n"
    comparison_report += res_comparison_note

    # Video Bitrate Comparison
    bitrate1_v = file1_details['video']['bitrate']
    bitrate2_v = file2_details['video']['bitrate']
    comparison_report += f"\nVideo Bitrate:\n  {file1_name}: {bitrate1_v} bps\n  {file2_name}: {bitrate2_v} bps\n"
    bitrate_v_comparison_note = ""
    if bitrate1_v != "N/A" and bitrate2_v != "N/A":
        if bitrate1_v > bitrate2_v:
            bitrate_v_comparison_note = f"  -> {file1_name} has a higher video bitrate. Higher bitrate often (but not always) indicates better quality due to less compression.\n"
        elif bitrate2_v > bitrate1_v:
            bitrate_v_comparison_note = f"  -> {file2_name} has a higher video bitrate. Higher bitrate often (but not always) indicates better quality due to less compression.\n"
        else:
            bitrate_v_comparison_note = f"  -> Video bitrates are the same.\n"
    else:
        bitrate_v_comparison_note = f"  -> Unable to compare video bitrate.\n"
    comparison_report += bitrate_v_comparison_note

    # Audio Bitrate Comparison
    bitrate1_a = file1_details['audio']['bitrate']
    bitrate2_a = file2_details['audio']['bitrate']
    comparison_report += f"\nAudio Bitrate:\n  {file1_name}: {bitrate1_a} bps\n  {file2_name}: {bitrate2_a} bps\n"
    bitrate_a_comparison_note = ""
    if bitrate1_a != "N/A" and bitrate2_a != "N/A":
        if bitrate1_a > bitrate2_a:
            bitrate_a_comparison_note = f"  -> {file1_name} has a higher audio bitrate, potentially better audio fidelity.\n"
        elif bitrate2_a > bitrate1_a:
            bitrate_a_comparison_note = f"  -> {file2_name} has a higher audio bitrate, potentially better audio fidelity.\n"
        else:
            bitrate_a_comparison_note = f"  -> Audio bitrates are the same.\n"
    else:
        bitrate_a_comparison_note = f"  -> Unable to compare audio bitrate.\n"
    comparison_report += bitrate_a_comparison_note

    # Scan Type Comparison (Progressive vs Interlaced)
    scan_type1 = file1_details['video']['scan_type']
    scan_type2 = file2_details['video']['scan_type']
    comparison_report += f"\nScan Type (Progressive/Interlaced):\n  {file1_name}: {scan_type1}\n  {file2_name}: {scan_type2}\n"
    scan_type_comparison_note = ""
    if scan_type1 != "N/A" and scan_type2 != "N/A" and scan_type1 != scan_type2:
        if scan_type1.lower() == 'progressive': # Assuming 'progressive' or similar string indicates progressive
            scan_type_comparison_note = f"  -> {file1_name} is Progressive, while {file2_name} is likely Interlaced. Progressive is generally preferred for modern displays.\n"
        elif scan_type2.lower() == 'progressive':
            scan_type_comparison_note = f"  -> {file2_name} is Progressive, while {file1_name} is likely Interlaced. Progressive is generally preferred for modern displays.\n"
        else:
            scan_type_comparison_note = f"  -> Scan types are different, but neither clearly Progressive. Check 'field_order' details for more info if available.\n"
    elif scan_type1 != "N/A" and scan_type2 != "N/A" and scan_type1 == scan_type2:
        scan_type_comparison_note = f"  -> Both files have the same scan type ({scan_type1}).\n"
    else:
        scan_type_comparison_note = f"  -> Unable to fully compare scan types.\n"
    comparison_report += scan_type_comparison_note


    # Pixel Format Comparison
    pixel_format1 = file1_details['video']['pixel_format']
    pixel_format2 = file2_details['video']['pixel_format']
    comparison_report += f"\nPixel Format:\n  {file1_name}: {pixel_format1}\n  {file2_name}: {pixel_format2}\n"
    pixel_format_comparison_note = ""
    if pixel_format1 != "N/A" and pixel_format2 != "N/A" and pixel_format1 != pixel_format2:
        if '444' in pixel_format1 and '420' in pixel_format2:
            pixel_format_comparison_note = f"  -> {file1_name} uses a '444' pixel format (like {pixel_format1}), which can preserve more color information than the '420' format of {file2_name} ({pixel_format2}). '444' is less common for distribution though.\n"
        elif '444' in pixel_format2 and '420' in file1_details['video']['pixel_format']: # Corrected to use file1_details
            pixel_format_comparison_note = f"  -> {file2_name} uses a '444' pixel format (like {pixel_format2}), which can preserve more color information than the '420' format of {file1_name} ({pixel_format1}). '444' is less common for distribution though.\n"
        else:
            pixel_format_comparison_note = f"  -> Pixel formats are different ({pixel_format1} vs {pixel_format2}). The impact on quality depends on the specific formats.\n"
    elif pixel_format1 != "N/A" and pixel_format2 != "N/A" and pixel_format1 == pixel_format2:
        pixel_format_comparison_note = f"  -> Both files use the same pixel format ({pixel_format1}).\n"
    else:
        pixel_format_comparison_note = f"  -> Unable to fully compare pixel formats.\n"
    comparison_report += pixel_format_comparison_note

    # Color Space, Range, Transfer (More advanced - for detailed analysis)
    color_details_report = "\nColor Details (for advanced analysis):\n"
    color_details_report += f"  {file1_name}:\n"
    color_details_report += f"    Color Space: {file1_details['video']['color_space']}, Color Range: {file1_details['video']['color_range']}, Color Transfer: {file1_details['video']['color_transfer']}\n"
    color_details_report += f"  {file2_name}:\n"
    color_details_report += f"    Color Space: {file2_details['video']['color_space']}, Color Range: {file2_details['video']['color_range']}, Color Transfer: {file2_details['video']['color_transfer']}\n"
    comparison_report += color_details_report

    # Variable Frame Rate Check
    vfr_report = "\nVariable Frame Rate (VFR) Check:\n"
    vfr_report += f"  {file1_name}: {'Variable Frame Rate' if file1_details['video']['is_variable_rate'] == '1' else 'Constant Frame Rate' if file1_details['video']['is_variable_rate'] == '0' else file1_details['video']['is_variable_rate']}\n"
    vfr_report += f"  {file2_name}: {'Variable Frame Rate' if file2_details['video']['is_variable_rate'] == '1' else 'Constant Frame Rate' if file2_details['video']['is_variable_rate'] == '0' else file2_details['video']['is_variable_rate']}\n"
    comparison_report += vfr_report


    # Video Codec
    codec1_v = file1_details['video']['codec'].lower()
    codec2_v = file2_details['video']['codec'].lower()
    comparison_report += f"\nVideo Codec:\n  {file1_name}: {codec1_v}\n  {file2_name}: {codec2_v}\n"

    # Audio Codec
    codec1_a = file1_details['audio']['codec'].lower()
    codec2_a = file2_details['audio']['codec'].lower()
    comparison_report += f"\nAudio Codec:\n  {file1_name}: {codec1_a}\n  {file2_name}: {codec2_a}\n"

    # Revised Overall Quality Suggestion:
    overall_suggestion = "\nOverall Quality Suggestion:\n"

    if res1 != "N/A" and res2 != "N/A":
        res1_pixels = int(res1.split('x')[0]) * int(res1.split('x')[1])
        res2_pixels = int(res2.split('x')[0]) * int(res2.split('x')[1])

        if res2_pixels > res1_pixels: # File 2 has higher resolution
            overall_suggestion += f"  {file2_name} has a significantly higher resolution than {file1_name}, which is a strong indicator of better visual quality.\n"
            if file2_details['overall_bitrate'] != "N/A" and file1_details['overall_bitrate'] != "N/A":
                if file2_details['overall_bitrate'] > file1_details['overall_bitrate']:
                    overall_suggestion += f"  Additionally, {file2_name} has a higher overall bitrate. Combined with the higher resolution, {file2_name} is **very likely the higher quality file**.\n"
                else:
                    overall_suggestion += f"  However, {file1_name} has a slightly higher or similar overall bitrate. While resolution favors {file2_name}, visual comparison is still recommended to confirm if the higher bitrate of {file1_name} compensates for the lower resolution.\n"
            else:
                overall_suggestion += f"  Overall bitrate comparison is inconclusive for one file. However, based on the significantly higher resolution of {file2_name}, it is **likely the higher quality file**.\n"

        elif res1_pixels > res2_pixels: # File 1 has higher resolution (less likely in this case based on previous outputs)
            overall_suggestion += f"  {file1_name} has a higher resolution than {file2_name}. This suggests potentially better visual quality.\n"
            if file2_details['overall_bitrate'] != "N/A" and file1_details['overall_bitrate'] != "N/A":
                if file1_details['overall_bitrate'] > file2_details['overall_bitrate']:
                    overall_suggestion += f"  Combined with a higher overall bitrate, {file1_name} is **likely the higher quality file**.\n"
                else:
                     overall_suggestion += f"  However, {file2_name} has a slightly higher or similar overall bitrate. While resolution favors {file1_name}, visual comparison is still recommended to confirm if the higher bitrate of {file2_name} compensates for the lower resolution.\n"
            else:
                overall_suggestion += f"  Overall bitrate comparison is inconclusive for one file. Based on higher resolution of {file1_name}, it is **likely the higher quality file**, but visual check recommended.\n"

        else: # Resolutions are the same or very similar
            overall_suggestion += "  Both files have similar resolutions.\n"
            if file2_details['overall_bitrate'] != "N/A" and file1_details['overall_bitrate'] != "N/A":
                if file2_details['overall_bitrate'] > file1_details['overall_bitrate']:
                    overall_suggestion += f"  {file2_name} has a significantly higher overall bitrate, making it **likely the slightly higher quality file**.\n"
                elif file1_details['overall_bitrate'] > file2_details['overall_bitrate']:
                    overall_suggestion += f"  {file1_name} has a significantly higher overall bitrate, making it **likely the slightly higher quality file**.\n"
                else:
                    overall_suggestion += "  Overall bitrates are also similar. Quality is likely comparable. Visual inspection is recommended to discern any subtle differences.\n"
            else:
                overall_suggestion += "  Overall bitrate comparison is inconclusive. Quality is likely comparable based on resolution and other similar metrics. Visual inspection is recommended.\n"
    else: # Resolution comparison not possible
        overall_suggestion += "  Resolution comparison is not possible.\n"
        if file2_details['overall_bitrate'] != "N/A" and file1_details['overall_bitrate'] != "N/A":
             if file2_details['overall_bitrate'] > file1_details['overall_bitrate']:
                    overall_suggestion += f"  {file2_name} has a higher overall bitrate, making it **potentially the slightly higher quality file**.\n"
             elif file1_details['overall_bitrate'] > file2_details['overall_bitrate']:
                    overall_suggestion += f"  {file1_name} has a higher overall bitrate, making it **potentially the slightly higher quality file**.\n"
             else:
                    overall_suggestion += "  Overall bitrates are also similar. Quality is difficult to determine without resolution data. Visual inspection is highly recommended.\n"
        else:
            overall_suggestion += "  Cannot definitively determine higher quality. Review the detailed metrics above for more information and perform visual inspection.\n"


    comparison_report += overall_suggestion
    comparison_report += "\n---------------------------------------------------\n"
    comparison_report += "**Important Notes:**\n"
    comparison_report += "* This analysis is based on technical metadata and provides an *indication* of potential quality differences.\n"
    comparison_report += "* **Perceived video quality is subjective and depends on many factors not captured here**, including encoding efficiency, source quality, personal preferences, and viewing conditions.\n"
    comparison_report += "* For a definitive quality assessment, **always visually compare the videos yourself**.\n"
    comparison_report += "* 'N/A' for bitrate or other values might mean the information was not readily available in the file's metadata, or the encoding method doesn't produce a fixed bitrate value.\n"

    return comparison_report


if __name__ == "__main__":
    # Replace with the actual paths to your video files
    file_path1 = "/mnt/e/Videos/top gear/Top_Gear_S08E08_400p_WEB-DL_30nama_30NAMA.mkv"
    file_path2 = "/mnt/e/Videos/top gear/Top Gear S08E08 - Vans.mp4"

    file1_details = get_video_quality_details(file_path1)
    file2_details = get_video_quality_details(file_path2)

    if file1_details:
        print("Details for File 1:")
        for key, value in file1_details.items():
            if key in ["video", "audio"]:
                print(f"  {key.capitalize()} Details:")
                for sub_key, sub_value in value.items():
                    print(f"    {sub_key.capitalize().replace('_', ' ')}: {sub_value}")
            else:
                print(f"  {key.capitalize().replace('_', ' ')}: {value}")
        print("-" * 30)

    if file2_details:
        print("\nDetails for File 2:")
        for key, value in file2_details.items():
            if key in ["video", "audio"]:
                print(f"  {key.capitalize()} Details:")
                for sub_key, sub_value in value.items():
                    print(f"    {sub_key.capitalize().replace('_', ' ')}: {sub_value}")
            else:
                print(f"  {key.capitalize().replace('_', ' ')}: {value}")
        print("-" * 30)

    if file1_details and file2_details:
        comparison_result = compare_video_quality(file1_details, file2_details)
        print(f"\n{comparison_result}")
    else:
        print("\nCould not retrieve details for one or both files to perform comparison.")

Details for File 1:
  Filename: Top_Gear_S08E08_400p_WEB-DL_30nama_30NAMA.mkv
  Format: matroska,webm
  Duration: 3571.615
  Overall bitrate: 1175821
  Video Details:
    Codec: h264
    Profile: High
    Resolution: 720x404
    Aspect ratio: 180:101
    Frame rate: 25/1
    Bitrate: N/A
    Scan type: progressive
    Pixel format: yuv420p
    Color space: N/A
    Color range: N/A
    Color transfer: N/A
    Is variable rate: N/A
  Audio Details:
    Codec: aac
    Channels: 2
    Sample rate: 48000
    Bitrate: N/A
------------------------------

Details for File 2:
  Filename: Top Gear S08E08 - Vans.mp4
  Format: mov,mp4,m4a,3gp,3g2,mj2
  Duration: 3557.935
  Overall bitrate: 2149033
  Video Details:
    Codec: h264
    Profile: Main
    Resolution: 1024x576
    Aspect ratio: 16:9
    Frame rate: 25/1
    Bitrate: 1949989
    Scan type: progressive
    Pixel format: yuv420p
    Color space: N/A
    Color range: N/A
    Color transfer: N/A
    Is variable rate: N/A
  Audio Details:
  